# SPR 2026 - Word2Vec + SVM

**Experimento Word2Vec com Support Vector Machine**

SVM é conhecido por funcionar bem com embeddings densos.

---
**CONFIGURAÇÃO OFFLINE:**
1. No Kaggle, vá em Settings → Internet → **OFF**
2. Gensim e scikit-learn já estão pré-instalados no Kaggle
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
import re
import warnings
warnings.filterwarnings('ignore')

SEED = 42
EMBEDDING_DIM = 100
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# Preprocessamento
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', text)
    return text.split()

train['tokens'] = train['report'].apply(preprocess)
test['tokens'] = test['report'].apply(preprocess)

print(f'Exemplo: {train["tokens"].iloc[0][:10]}')

In [ ]:
# Treinar Word2Vec
all_texts = train['tokens'].tolist() + test['tokens'].tolist()

w2v = Word2Vec(
    sentences=all_texts,
    vector_size=EMBEDDING_DIM,
    window=5,
    min_count=2,
    workers=4,
    epochs=10,
    seed=SEED
)

print(f'Vocabulário: {len(w2v.wv)} palavras')

In [ ]:
# Converter textos em embeddings (Mean Pooling)
def text_to_embedding(tokens, model, dim):
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

X_train = np.array([text_to_embedding(t, w2v, EMBEDDING_DIM) for t in train['tokens']])
X_test = np.array([text_to_embedding(t, w2v, EMBEDDING_DIM) for t in test['tokens']])
y_train = train['target'].values

print(f'X_train shape: {X_train.shape}')

In [ ]:
# Normalizar features (importante para SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Features normalizadas!')

In [ ]:
# Treinar SVM
model = SVC(
    C=1.0,
    kernel='rbf',
    gamma='scale',
    class_weight='balanced',
    random_state=SEED
)

model.fit(X_train_scaled, y_train)
print('Modelo SVM treinado!')

In [ ]:
# Predições e submissão
predictions = model.predict(X_test_scaled)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())